In [1]:
! pip install konlpy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ----------- ---------------------------- 5.5/19.4 MB 27.9 MB/s eta 0:00:01
   ---------------------------- ----------- 13.6/19.4 MB 34.2 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 34.0 MB/s  0:00:00

   ---------------------------------------- 0/2 [JPype1]
   ---------------------------------------- 0/2 [JPype1]
   ---------------------------------------- 0/2 [JPype1]
   ---------------------------------------- 0/2 [JPype1]
   ---------------------------------------- 0/2 [JPype1]
   ---------------------------------------- 0/2 [JPype1]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   ---------------------------------------- 2/2 [konlpy]



In [1]:
from konlpy.tag import Okt
from collections import Counter
import pandas as pd

In [2]:
location_by_district = {
    "남구": [
        "부산항대교", "부산박물관", "UN기념공원", 
        "오륙도스카이워크", "이기대도시자연공원", "용호만유람선부두"
    ],
    "기장군": [
        "일광해수욕장", "야구등대", "임랑해수욕장", "부산프리미엄아울렛", 
        "장안사", "안데르센동화마을", "아홉산숲", "부산치유의숲"
    ],
    "서구": [
        "송도해수욕장", "송도용궁구름다리", "동아대석당박물관", "남항대교", 
        "구덕운동장", "구덕야영장"
    ],
    "동구": [
        "상해거리", "초량이바구길"
    ],
    "수영구": [
        "광안대교", "광안리SUPZONE"
    ]
}

In [3]:
location_by_district = {
    # "해운대구": [
    #     "누리마루", "더베이101", "벡스코", "송정해수욕장", "영화의전당", 
    #     "장산", "해운대블루라인파크", "해운대해수욕장"
    # ],
    # "중구": [
    #     "국제시장", "보수동책방골목", "비프광장", "용두산공원", "자갈치시장"
    # ],
    # "부산진구": [
    #     "부전마켓타운", "성지곡수원지", "송상현광장", "전포카페거리", "황령산", "호천마을"
    # ],
    # "동래구": [
    #     "금강공원", "동래온천", "동래읍성"
    # ],
    # "북구": [
    #     "구포시장", "문화빙상센터", "부산어촌민속관", "화명생태공원"
    # ],
    "남구": [
        "부산항대교", "부산박물관", "UN기념공원", 
        "오륙도스카이워크", "이기대도시자연공원", "용호만유람선부두"
    ],
    # "영도구": [
    #     "태종대", "아미르공원", "절영해안산책로", "국립해양박물관", 
    #     "흰여울문화마을", "깡깡이예술마을", "영도대교"
    # ],
    # "사하구": [
    #     "감천문화마을", "아미산전망대", "다대포해수욕장", "다대포꿈의낙조분수", 
    #     "낙동강하구에코센터", "을숙도생태공원", "을숙도둘레길"
    # ],
    # "강서구": [
    #     "부네치아", "맥도생태공원", "강서체육공원", "대저생태공원"
    # ],
    "기장군": [
        "일광해수욕장", "야구등대", "임랑해수욕장", "부산프리미엄아울렛", 
        "장안사", "안데르센동화마을", "아홉산숲", "부산치유의숲"
    ],
    "서구": [
        "송도해수욕장", "송도용궁구름다리", "동아대석당박물관", "남항대교", 
        "구덕운동장", "구덕야영장"
    ],
    "동구": [
        "상해거리", "초량이바구길"
    ],
    "수영구": [
        "광안대교", "광안리SUPZONE"
    ],
    # "금정구": [
    #     "회동수원지", "서동미로시장", "금정산성", "범어사"
    # ],
    # "사상구": [
    #     "삼락생태공원", "사상인디스테이션"
    # ]
}

In [ ]:
import pandas as pd
import os
import re
from konlpy.tag import Okt
from collections import Counter

# 경로 설정
base_path = "/content/drive/MyDrive/빅데이터_부산/텍스트(네이버 리뷰)"
okt = Okt()

# [업데이트] 보내주신 결과를 바탕으로 정교하게 짠 불용어 리스트
stopwords= [
    # 1. 지역 상위어 (부산 내 장소 비교 시 변별력을 완전히 파괴하는 주범)
    '부산', '부산시', '부산여행', '부산가볼만한곳', '부산항', '부경', '경남', '부산역',
    
    # 2. 타 지역 지명 오염 (구덕야영장, 서동미로시장 등에서 발견된 오염 데이터 차단)
    '경기도', '수원', '인천', '부여', '원주', '방콕', '제주', '강원도', '망원', '잠원', '서울', '경기',
    
    # 3. 비유적 표현 및 중의성 단어 (아미르공원 '한강' 비유 등 분석 반영)
    '한강', '한국', '세계', '국내', '다른', '바로', '진짜', '어디', '현지', '이후',
    
    '여행', '사진', '사진가', '모습', '생각', '정도', '오늘', '이번', '추천', '인기', '방문', 
    '정보', '위치', '문화', '건물', '관련', '보고', '정말', '준비', '시작', '다양한', '함께',
    
    # 6. 수치/행위 및 단위 (데이터 정교화 과정에서 발견)
    '인분', '주문', '메뉴', '가격', '체험', '관람', '이용', '내용', '상황', '확인'
]


def analyze_and_print_text():
    for district, places in location_by_district.items():
        print(f"\n" + "="*50)
        print(f"📂 [구역: {district}] 최종 정밀 분석")
        print("="*50)
        
        for place in places:
            csv_path = os.path.join(base_path, district, f"naver_blog_{place}_stable.csv")
            xlsx_path = os.path.join(base_path, district, f"naver_blog_{place}_stable.xlsx")
            
            df = None
            if os.path.exists(csv_path):
                try:
                    df = pd.read_csv(csv_path, encoding='utf-8-sig', on_bad_lines='skip', quotechar='"', skipinitialspace=True)
                except:
                    df = pd.read_csv(csv_path, encoding='cp949', on_bad_lines='skip', quotechar='"')
            elif os.path.exists(xlsx_path):
                df = pd.read_excel(xlsx_path)

            if df is None or df.empty: continue

            try:
                cols = df.columns.tolist()
                content_col = next((c for c in cols if '본문' in c), cols[2] if len(cols) > 2 else None)
                title_col = next((c for c in cols if '제목' in c), cols[0] if len(cols) > 0 else None)
                
                # 제목 3배 가중치 + 본문 합치기
                full_text = " ".join((df[title_col].astype(str) + " ") * 3 + df[content_col].astype(str))
                clean_text = re.sub(r'[^가-힣\s]', ' ', full_text)
                
                # 명사 추출
                nouns = okt.nouns(clean_text)
                
                # [핵심] 불용어 제거 및 2글자 이상만 추출
                # 또한 장소 이름 자체가 키워드 1위인 경우가 많으므로 필요한 경우 장소 이름도 필터링 가능
                final_nouns = [n for n in nouns if len(n) > 1 and n not in stopwords]
                
                count = Counter(final_nouns)
                top_30 = count.most_common(30)
                
                print(f"\n📍 {place} 핵심 키워드 TOP 30")
                print("-" * 40)
                for i, (word, freq) in enumerate(top_30, 1):
                    # 가독성을 위해 3열로 출력
                    print(f"{word:<12}({freq:>4}회)", end='\t')
                    if i % 3 == 0: print()
                print()
                
            except Exception as e:
                print(f"❌ {place} 분석 실패: {e}")

analyze_and_print_text()

FileNotFoundError: [Errno 2] No such file or directory: 'your_file.csv'